# Menjalankan skrip pelatihan dengan Python SDK

Anda dapat menggunakan Python SDK untuk Azure Machine Learning untuk mengirimkan skrip sebagai pekerjaan. Dengan menggunakan pekerjaan, Anda dapat dengan mudah melacak parameter input dan output saat melatih model pembelajaran mesin.

## Sebelum Anda memulai

Anda memerlukan versi terbaru paket **azureml-ai-ml** untuk menjalankan kode di notebook ini. Jalankan sel di bawah ini untuk memverifikasi bahwa sel diinstal.

> **Catatan:**
> Jika paket **azure-ai-ml** tidak diinstal, jalankan `pip install azure-ai-ml` untuk menginstalnya.

In [ ]:
## Menyambungkan ke ruang kerja Anda

Dengan paket SDK yang diperlukan terinstal, sekarang Anda siap untuk terhubung ke ruang kerja Anda.

Untuk menyambungkan ke ruang kerja, kita memerlukan parameter pengidentifikasi - ID langganan, nama grup sumber daya, dan nama ruang kerja. Nama grup sumber daya dan nama ruang kerja sudah diisi untuk Anda. Anda hanya memerlukan ID langganan untuk menyelesaikan perintah.

Untuk menemukan parameter yang diperlukan, klik nama langganan dan ruang kerja di kanan atas Studio. Panel akan terbuka di sebelah kanan.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Salin ID langganan dan ganti **YOUR-SUBSCRIPTION-ID** dengan nilai yang Anda salin. </p>

## Menggunakan Python SDK untuk melatih model

Untuk melatih model, Anda akan terlebih dahulu membuat skrip **diabetes_training.py** di folder **src** . Skrip menggunakan file **diabetes.csv** di folder yang sama dengan data pelatihan.

In [ ]:
Jalankan sel di bawah ini untuk mengirimkan pekerjaan yang melatih model klasifikasi untuk memprediksi diabetes. 

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Use the Python SDK to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))


Run the cell below to submit the job that trains a classification model to predict diabetes. 

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-pythonv2-train",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)